In [1]:
from docplex.mp.model import Model
import numpy as np
from sklearn.linear_model import LinearRegression
import docplex.mp.conflict_refiner as cr

In [2]:
def galo_bids_init(value_model, bidder_id, n, presampled_n, presampled_algorithm, ml_model):
    np.random.seed(0)
    values = np.random.randint(0, 100, size=(10)).tolist()
    bundles = np.random.randint(0, 2, size=(10, 5)).tolist()
    #print(bundles)
    M = len(bundles[0])

    # bundles random 18 bundles of length 10
    reg = LinearRegression().fit(bundles, values)

    count = 10
    bigM = 300
    while count <= 30:
        reg = LinearRegression().fit(bundles, values)
        coef = reg.coef_
        intercept = reg.intercept_
        print(coef)
        print(intercept)
        x_vectors = []
        x_distances = []
        for bundle in range(len(bundles)):

            m = Model(name='galo')

            r = m.continuous_var(name='r')
            
            x = m.binary_var_list(range(M), name='x')
            
            b = [m.binary_var(name=f'b_{j}') for j in range(len(values))]

            constraints = []

            y_prediction = m.sum(x[k]*coef[k] for k in range(M)) + intercept

            constraints.append(values[bundle] - y_prediction <= r)
            constraints.append(y_prediction - values[bundle] <= r)

            for j in range(len(values)):
                if j == bundle:
                    continue
                constraints.append(values[j] - y_prediction + bigM * b[j] >= r)
                constraints.append(y_prediction - values[j] + bigM * (1 - b[j]) >= r)

            m.add_constraints(constraints)
            
            m.maximize(r)
            
            sol = m.solve()

            res = m.get_solve_status()

            if res.name == 'INFEASIBLE_SOLUTION':  # or also 'INFEASIBLE_OR_UNBOUNDED_SOLUTION'
                cref = cr.ConflictRefiner()
                print('show some of the constraints that can be removed to arrive at a minimal conflict')
                confl = cref.refine_conflict(m, display=True)  # display flag is to show the conflicts
                print('Initial y')
                print(values[bundle])
                print('Intercept')
                print(intercept)
                cref.display_conflicts(confl)

            try:
                print(sol.solve_status)

            except:
                print('No solution found')
                continue

            vector = np.array([x[k].solution_value for k in range(M)])
            x_vectors.append(vector)
            distance = np.sum(np.abs(np.sum(vector * coef) + intercept - values[bundle]))
            x_distances.append(distance)

        chosen_bundle = x_vectors[np.argmax(x_distances)]
        #generate one random value
        value = np.random.randint(0, 100)
        values.append(value)
        bundles.append(chosen_bundle.tolist())
        count+=1

    return bundles

results = galo_bids_init(None, 0, 20, 10, 'unif', None)
#results

[  6.88679245  38.73584906  -2.88679245   3.8490566  -18.86792453]
35.58490566037738
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
[  7.15517241  38.5         -3.15517241   3.77586207 -18.96551724]
35.70689655172416
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
[ 16.92384106  34.00331126 -12.92384106  -1.3410596   -7.8013245 ]
35.24172185430466
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveS

/home/artem/.conda/envs/ica_new/lib/python3.6/site-packages/docplex/mp/model.py:3278: UserWarning: Unknown quick-setting in Context: display, value: True
  context.update_key_value(argname, argval)


JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
[ -1.55973154  13.05100671  -4.50604027  18.88590604 -21.73557047]
37.08053691275167
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
show some of the constraints that can be removed to arrive at a minimal conflict
Initial y
83
Intercept
37.08053691275167
Conflict set:
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 1.560x_0-13.051x_1+4.506x_2-18.886x_3+21.736x_4+45.919 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 1.560x_0-13.051x_1+4.506x_2-18.886x_3+21.736x_4+300b_2+26.919 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -1.560x_0+13.051x_1-4.506x_2+18.886x_3-21.736x_4-300b_2+273.081 >= r
No solution found


/home/artem/.conda/envs/ica_new/lib/python3.6/site-packages/docplex/mp/model.py:3278: UserWarning: Unknown quick-setting in Context: display, value: True
  context.update_key_value(argname, argval)


JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
show some of the constraints that can be removed to arrive at a minimal conflict


...

Initial y
55
Intercept
27.925116927057424
Conflict set:
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.160x_0-15.864x_1+3.751x_2-24.356x_3+20.984x_4+27.075 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.160x_0+15.864x_1-3.751x_2+24.356x_3-20.984x_4-27.075 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.160x_0-15.864x_1+3.751x_2-24.356x_3+20.984x_4+300b_2+36.075 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.160x_0+15.864x_1-3.751x_2+24.356x_3-20.984x_4-300b_2+263.925 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.160x_0-15.864x_1+3.751x_2-24.356x_3+20.984x_4+300b_16+25.075 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.160x_0+15.864x_1-3.751x_2+24.356x_3-20.984x_4-300b_16+274.925 >= r
No solution found

KeyboardInterrupt
JobSolve

/home/artem/.conda/envs/ica_new/lib/python3.6/site-packages/docplex/mp/model.py:3278: UserWarning: Unknown quick-setting in Context: display, value: True
  context.update_key_value(argname, argval)


Initial y
0
Intercept
27.925116927057424
Conflict set:
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.160x_0+15.864x_1-3.751x_2+24.356x_3-20.984x_4+27.925 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.160x_0-15.864x_1+3.751x_2-24.356x_3+20.984x_4+300b_17-22.925 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.160x_0+15.864x_1-3.751x_2+24.356x_3-20.984x_4-300b_17+322.925 >= r
No solution found
show some of the constraints that can be removed to arrive at a minimal conflict


/home/artem/.conda/envs/ica_new/lib/python3.6/site-packages/docplex/mp/model.py:3278: UserWarning: Unknown quick-setting in Context: display, value: True
  context.update_key_value(argname, argval)


Initial y
0
Intercept
27.925116927057424
Conflict set:
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.160x_0+15.864x_1-3.751x_2+24.356x_3-20.984x_4+27.925 <= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): 0.160x_0-15.864x_1+3.751x_2-24.356x_3+20.984x_4+300b_17-22.925 >= r
	<class 'docplex.mp.constr.LinearConstraint'> (status code: ConflictStatus.Member): -0.160x_0+15.864x_1-3.751x_2+24.356x_3-20.984x_4-300b_17+322.925 >= r
No solution found
No solution found
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
[  0.72921424  18.36064473  -2.17454668  22.261182   -21.44996642]
25.837676292813953
